<a href="https://colab.research.google.com/github/ccosmin97/idc-prostate-mri-analysis/blob/main/analysis_statistical_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports


In [1]:
!pip install scikit_posthocs

In [40]:
import os
import numpy as np
import glob
import pandas as pd
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import pylab
from scipy.stats import wilcoxon

In [3]:
import plotly.graph_objects as go
import plotly.express as px
import os
import plotly.io as pio
import scikit_posthocs as sp

In [4]:
from numpy.random import seed
from numpy.random import randn
from scipy.stats import shapiro
from scipy.stats import friedmanchisquare

In [5]:
# from IPython.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

In [6]:
#colab
from google.colab import auth
auth.authenticate_user()
project_id = "idc-sandbox-003"
os.environ["GCP_PROJECT_ID"] = project_id
from google.cloud import bigquery
bq_client = bigquery.Client(os.environ["GCP_PROJECT_ID"])

# Import analysis data

In [ ]:
# !wget https://github.com/nytimes/covid-19-data/archive/refs/heads/master.zip

In [24]:
selection_query = f"""
SELECT *
FROM `idc-sandbox-003.prostate_benchmarking_neurips_2024.prostate_benchmark_simple_table_all_table`
 """
selection_result = bq_client.query(selection_query)
data_df = selection_result.result().to_dataframe()
data_df["asd"] = data_df.asd.replace(-999,np.NaN)
data_df = data_df.astype({'expert_Value': 'float32', 'ai_Value': 'float32'})
data_df["expert_Value"] = data_df.expert_Value.apply(lambda x : x/1000)#change Volume of segment from mm3 to mL
data_df["ai_Value"] = data_df.ai_Value.apply(lambda x : x/1000 if x is not None else x)#change Volume of segment from mm3 to mL
#round AI and expert quantitative results to 0.1 and .2 decimals
for field_to_round in ["ai_Value", "expert_Value"]:
  na_mask = data_df[field_to_round].notnull()
  data_df.loc[na_mask, "ai_Value"] = data_df.loc[na_mask, field_to_round].astype(float).round(1)
data_df = data_df.round({'dsc': 2, 'hsdff': 2, 'hsdff_95': 2,
                    'asd': 2, 'prediction_time': 1})

# Analysis


## Seaborn parameters


In [8]:
sns.set(rc={'figure.figsize':(10,10)})
# Setting the font size by 2.5
sns.set(font_scale=1.5)

## Mean and STDs of quantitative region and distance based metrics


In [9]:
data_df.algorithmNameCollection.unique()

array(['prostate158-prostate_mri_us_biopsy-prostate',
       'nnunet_task024-prostate_mri_us_biopsy-prostate',
       'prostate158-qin_prostate_repeatability-prostate',
       'nnunet_task05-qin_prostate_repeatability-prostate',
       'nnunet_task024-qin_prostate_repeatability-prostate',
       'prostate158-prostatex-zonal', 'prostate158-prostatex-prostate',
       'nnunet_task05-prostatex-prostate',
       'nnunet_task05-prostatex-zonal',
       'nnunet_task024-prostatex-prostate'], dtype=object)

### ProstateX analysis


#### Whole prostate


In [25]:
prostate_prostatex_df = data_df[(data_df.collection_id == 'prostatex') & (data_df.segment == 'Prostate')]
print(len(prostate_prostatex_df))

198


In [26]:
nnnunet_task024_prostate_df = prostate_prostatex_df[prostate_prostatex_df.algorithmNameCollection == 'nnunet_task024-prostatex-prostate']
nnnunet_task05_prostate_df = prostate_prostatex_df[prostate_prostatex_df.algorithmNameCollection == 'nnunet_task05-prostatex-prostate']
monai_prostate158_prostate_df = prostate_prostatex_df[prostate_prostatex_df.algorithmNameCollection == 'prostate158-prostatex-prostate']

In [27]:
# pd.set_option('display.width', -1)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

##### Quantitative metrics summary for ProstateX whole prostate segment


In [29]:
nnnunet_task024_prostate_df[['dsc', 'hsdff', 'hsdff_95', 'asd', 'ai_Value', 'expert_Value']].round(2).describe(include = 'all')

,dsc,hsdff,hsdff_95,asd,ai_Value,expert_Value
count,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000
mean,0.922879,6.698030,0.915758,0.779242,59.024242,59.027273
std,0.025526,3.495229,0.629840,0.304304,29.637575,29.643197
min,0.850000,3.320000,0.000000,0.350000,17.600000,17.600000
25%,0.910000,4.660000,0.500000,0.522500,38.825000,38.815000
50%,0.930000,6.000000,0.750000,0.735000,53.900000,53.910000
75%,0.940000,6.930000,1.350000,0.947500,73.525000,73.562500
max,0.960000,24.230000,3.970000,1.650000,161.900000,161.940000


In [30]:
nnnunet_task05_prostate_df[['dsc', 'hsdff', 'hsdff_95', 'asd', 'ai_Value', 'expert_Value']].round(2).describe(include = 'all')

,dsc,hsdff,hsdff_95,asd,ai_Value,expert_Value
count,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000
mean,0.895606,11.689091,1.703182,1.228636,59.024242,59.027273
std,0.037953,13.187189,1.082287,0.650015,29.637575,29.643197
min,0.690000,4.030000,0.250000,0.600000,17.600000,17.600000
25%,0.870000,6.100000,1.010000,0.865000,38.825000,38.815000
50%,0.900000,7.430000,1.510000,1.050000,53.900000,53.910000
75%,0.920000,9.010000,2.000000,1.300000,73.525000,73.562500
max,0.950000,66.070000,6.670000,3.640000,161.900000,161.940000


In [34]:
monai_prostate158_prostate_df[['dsc', 'hsdff', 'hsdff_95', 'asd', 'ai_Value', 'expert_Value']].round(2).describe(include = 'all')

,dsc,hsdff,hsdff_95,asd,ai_Value,expert_Value
count,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000
mean,0.902424,8.286818,1.931515,1.490606,59.024242,59.027273
std,0.076421,13.587150,6.891053,5.033204,29.637575,29.643197
min,0.310000,3.910000,0.350000,0.490000,17.600000,17.600000
25%,0.900000,5.910000,0.750000,0.730000,38.825000,38.815000
50%,0.910000,6.020000,1.045000,0.850000,53.900000,53.910000
75%,0.927500,7.305000,1.490000,0.997500,73.525000,73.562500
max,0.940000,116.230000,56.970000,41.720000,161.900000,161.940000


Normality tests


In [35]:
for df_val in [nnnunet_task024_prostate_df,nnnunet_task05_prostate_df,monai_prostate158_prostate_df]:
    # normality test
    stat, p = shapiro(df_val.dsc.values)
    print('Statistics=%.3f, p=%.3f' % (stat, p))
    # interpret
    alpha = 0.05
    if p > alpha:
        print('Sample looks Gaussian (fail to reject H0)')
    else:
        print('Sample does not look Gaussian (reject H0)')
    print("\n")

Statistics=0.906, p=0.000
Sample does not look Gaussian (reject H0)


Statistics=0.797, p=0.000
Sample does not look Gaussian (reject H0)


Statistics=0.285, p=0.000
Sample does not look Gaussian (reject H0)




Mean and STD deviations for prostatex whole prostate results


In [37]:
print(f"nnunet task024 model DSC mean and std for whole prostate: {round(np.mean(nnnunet_task024_prostate_df.dsc.values),2)}/{round(np.std(nnnunet_task024_prostate_df.dsc.values),2)}")
print(f"nnunet task05 model DSC mean and std for whole prostate: {round(np.mean(nnnunet_task05_prostate_df.dsc.values),2)}/{round(np.std(nnnunet_task05_prostate_df.dsc.values),2)}")
print(f"nnunet monai prostate158 DSC model mean and std for whole prostate: {round(np.mean(monai_prostate158_prostate_df.dsc.values),2)}/{round(np.std(monai_prostate158_prostate_df.dsc.values),2)}")

nnunet task024 model DSC mean and std for whole prostate: 0.92/0.03
nnunet task05 model DSC mean and std for whole prostate: 0.9/0.04
nnunet monai prostate158 DSC model mean and std for whole prostate: 0.9/0.08


##### Paired tests on mean DSC, same collection == Wilcoxon signed rank test with Bonferroni correction


The Wilcoxon signed-rank test tests the null hypothesis that two related paired samples come from the same distribution.

In particular, it tests whether the distribution of the differences x - y is symmetric about zero.

It is a non-parametric version of the paired T-test.


The p-value measures the probability of getting a more extreme value than the one you got from the experiment.

If the p-value is greater than alpha, you accept the null hypothesis.

If it is less than alpha, you reject the null hypothesis.


**Bonferroni correction : 3 paired Wilcoxon tests so alpha = 0.05/3 = 0.017**


Pair 1 : task024 dsc differences vs task05


In [45]:
diff = nnnunet_task024_prostate_df.sort_values(by=['refSerieUID']).dsc.values - nnnunet_task05_prostate_df.sort_values(by=['refSerieUID']).dsc.values

In [46]:
res = wilcoxon(diff)
res.statistic, res.pvalue

(20.0, 6.07257483705153e-11)

In [47]:
res = wilcoxon(diff, alternative='less')
res.statistic, res.pvalue

(1750.0, 0.9999999999696372)

This shows that the null hypothesis that the median of dsc differences is negative can be rejected at a confidence level of 5% in favor of the alternative that the median is greater than zero.


Pair 2 : task024 dsc differences vs monai


In [48]:
diff = nnnunet_task024_prostate_df.sort_values(by=['refSerieUID']).dsc.values - monai_prostate158_prostate_df.sort_values(by=['refSerieUID']).dsc.values
res = wilcoxon(diff)
res.statistic, res.pvalue

(325.0, 6.298514251178753e-05)

In [49]:
res = wilcoxon(diff, alternative='less')
res.statistic, res.pvalue

(1328.0, 0.9999685074287441)

Pair 3 : task05 dsc differences vs monai


In [50]:
diff = monai_prostate158_prostate_df.sort_values(by=['refSerieUID']).dsc.values - nnnunet_task05_prostate_df.sort_values(by=['refSerieUID']).dsc.values
res = wilcoxon(diff)
res.statistic, res.pvalue

(320.0, 0.0001498725787900786)

In [51]:
res = wilcoxon(diff, alternative='greater')
res.statistic, res.pvalue

(1220.0, 7.49362893950393e-05)

#### Peripheral Zone


In [69]:
pz_prostatex_df = data_df[(data_df.collection_id == 'prostatex') & (data_df.segment.str.contains('Peripheral'))]

In [70]:
pz_prostatex_df.algorithmNameCollection.unique()

array(['prostate158-prostatex-zonal', 'nnunet_task05-prostatex-zonal'],
      dtype=object)

In [71]:
nnunet_pz_prostatex_df =  pz_prostatex_df[pz_prostatex_df.algorithmNameCollection == 'nnunet_task05-prostatex-zonal']
monai_pz_prostatex_df = pz_prostatex_df[pz_prostatex_df.algorithmNameCollection == 'prostate158-prostatex-zonal']

In [72]:
nnunet_pz_prostatex_df = nnunet_pz_prostatex_df.drop_duplicates(subset='refSerieUID', keep="last")

##### Quantitative metrics summary for ProstateX peripheral zone segment

In [73]:
nnunet_pz_prostatex_df[['dsc', 'hsdff', 'hsdff_95', 'asd', 'ai_Value', 'expert_Value']].round(2).describe(include = 'all')

,dsc,hsdff,hsdff_95,asd,ai_Value,expert_Value
count,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000
mean,0.667857,17.397755,5.005918,2.334184,19.774490,19.778776
std,0.155837,7.703612,3.670226,3.454579,6.978824,6.976135
min,0.000000,7.430000,1.500000,0.610000,6.500000,6.550000
25%,0.632500,11.562500,2.682500,1.012500,14.950000,14.912500
50%,0.710000,15.635000,3.840000,1.395000,18.700000,18.740000
75%,0.760000,20.967500,5.962500,2.327500,23.050000,23.062500
max,0.840000,46.130000,21.400000,27.670000,42.100000,42.050000


In [74]:
monai_pz_prostatex_df[['dsc', 'hsdff', 'hsdff_95', 'asd', 'ai_Value', 'expert_Value']].round(2).describe(include = 'all')

,dsc,hsdff,hsdff_95,asd,ai_Value,expert_Value
count,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000
mean,0.730510,13.146327,3.383571,1.145816,19.774490,19.778776
std,0.083213,5.526158,2.058936,0.561258,6.978824,6.976135
min,0.410000,4.640000,1.000000,0.470000,6.500000,6.550000
25%,0.700000,9.392500,2.055000,0.762500,14.950000,14.912500
50%,0.750000,11.970000,2.710000,1.040000,18.700000,18.740000
75%,0.787500,16.160000,4.190000,1.347500,23.050000,23.062500
max,0.900000,33.980000,15.040000,4.140000,42.100000,42.050000


In [75]:
print(f"nnunet task05 model DSC mean and std for pz: {round(np.mean(nnunet_pz_prostatex_df.dsc.values),2)}/{round(np.std(nnunet_pz_prostatex_df.dsc.values),2)}")
print(f"monai model DSC mean and std for pz: {round(np.mean(monai_pz_prostatex_df.dsc.values),2)}/{round(np.std(monai_pz_prostatex_df.dsc.values),2)}")

nnunet task05 model DSC mean and std for pz: 0.67/0.16
monai model DSC mean and std for pz: 0.73/0.08


##### Paired tests on mean DSC, same collection == Wilcoxon signed rank test


In [61]:
diff = monai_pz_prostatex_df.sort_values(by=['refSerieUID']).dsc.values - nnunet_pz_prostatex_df.sort_values(by=['refSerieUID']).dsc.values
res = wilcoxon(diff)
res.statistic, res.pvalue

(834.0, 2.1865286957299772e-07)

In [62]:
res = wilcoxon(diff, alternative='greater')
res.statistic, res.pvalue

(3537.0, 1.0932643478649886e-07)

#### Transition zone


In [63]:
tz_prostatex_df = data_df[(data_df.collection_id == 'prostatex') & (data_df.segment.str.contains('Transition'))]

In [64]:
nnunet_tz_prostatex_df =  tz_prostatex_df[tz_prostatex_df.algorithmNameCollection == 'nnunet_task05-prostatex-zonal']
monai_tz_prostatex_df = tz_prostatex_df[tz_prostatex_df.algorithmNameCollection == 'prostate158-prostatex-zonal']

In [65]:
nnunet_tz_prostatex_df = nnunet_tz_prostatex_df.drop_duplicates(subset='refSerieUID', keep="last")

##### Quantitative metrics summary for ProstateX Transition zone segment

In [76]:
nnunet_tz_prostatex_df[['dsc', 'hsdff', 'hsdff_95', 'asd', 'ai_Value', 'expert_Value']].round(2).describe(include = 'all')

,dsc,hsdff,hsdff_95,asd,ai_Value,expert_Value
count,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000
mean,0.817857,15.785306,3.099286,2.050510,45.190816,45.192041
std,0.112555,14.497108,2.857834,1.443969,30.083136,30.085658
min,0.330000,5.230000,0.750000,0.800000,6.100000,6.130000
25%,0.780000,9.272500,1.750000,1.282500,22.400000,22.385000
50%,0.850000,10.630000,2.290000,1.640000,36.600000,36.570000
75%,0.890000,13.460000,3.500000,2.167500,60.025000,60.032500
max,0.930000,67.770000,21.800000,10.950000,144.100000,144.140000


In [77]:
monai_tz_prostatex_df[['dsc', 'hsdff', 'hsdff_95', 'asd', 'ai_Value', 'expert_Value']].round(2).describe(include = 'all')

,dsc,hsdff,hsdff_95,asd,ai_Value,expert_Value
count,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000
mean,0.845510,11.436837,3.487245,2.571531,45.190816,45.192041
std,0.135503,12.741837,11.692521,10.154334,30.083136,30.085658
min,0.000000,4.640000,0.500000,0.630000,6.100000,6.130000
25%,0.830000,7.682500,1.445000,0.970000,22.400000,22.385000
50%,0.880000,9.390000,1.750000,1.150000,36.600000,36.570000
75%,0.900000,11.877500,2.362500,1.362500,60.025000,60.032500
max,0.940000,119.750000,102.930000,90.040000,144.100000,144.140000


In [78]:
print(f"nnunet task05 model DSC mean and std for tz: {round(np.mean(nnunet_tz_prostatex_df.dsc.values),2)}/{round(np.std(nnunet_tz_prostatex_df.dsc.values),2)}")
print(f"monai model DSC mean and std for tz: {round(np.mean(monai_tz_prostatex_df.dsc.values),2)}/{round(np.std(monai_tz_prostatex_df.dsc.values),2)}")

nnunet task05 model DSC mean and std for tz: 0.82/0.11
monai model DSC mean and std for tz: 0.85/0.13


##### Paired tests on mean DSC, same collection == Wilcoxon signed rank test


In [79]:
diff = monai_tz_prostatex_df.sort_values(by=['refSerieUID']).dsc.values - nnunet_tz_prostatex_df.sort_values(by=['refSerieUID']).dsc.values
res = wilcoxon(diff)
res.statistic, res.pvalue

(474.5, 5.525883664815393e-11)

In [80]:
res = wilcoxon(diff, alternative='greater')
res.statistic, res.pvalue

(474.5, 0.9999999999723705)

### QIN collection


#### Whole prostate


In [81]:
prostate_qin_df = data_df[(data_df.collection_id == 'qin_prostate_repeatability') & (data_df.segment == 'Prostate')]
prostate_qin_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 90 entries, 1623 to 1771
Data columns (total 59 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   segmentationInstanceUID               90 non-null     object 
 1   StudyInstanceUID                      90 non-null     object 
 2   sourceSegmentedSeriesUID              90 non-null     object 
 3   expert_QuantityCodeMeaning            90 non-null     object 
 4   expert_QuantityCodeValue              90 non-null     object 
 5   expert_Value                          90 non-null     float64
 6   expert_UnitsCodeValue                 90 non-null     object 
 7   expert_UnitsCodeMeaning               90 non-null     object 
 8   expert_findingSiteCodeValue           90 non-null     object 
 9   expert_findingSiteCodeMeaning         90 non-null     object 
 10  ai_QuantityCodeMeaning                90 non-null     object 
 11  ai_QuantityCodeValue 

In [83]:
prostate_qin_df.algorithmNameCollection.unique()

array(['prostate158-qin_prostate_repeatability-prostate',
       'nnunet_task05-qin_prostate_repeatability-prostate',
       'nnunet_task024-qin_prostate_repeatability-prostate'], dtype=object)

In [84]:
nnnunet_task024_prostate_qin_df = prostate_qin_df[prostate_qin_df.algorithmNameCollection == 'nnunet_task024-qin_prostate_repeatability-prostate']
nnnunet_task05_prostate_qin_df = prostate_qin_df[prostate_qin_df.algorithmNameCollection == 'nnunet_task05-qin_prostate_repeatability-prostate']
monai_prostate158_prostate_qin_df = prostate_qin_df[prostate_qin_df.algorithmNameCollection == 'prostate158-qin_prostate_repeatability-prostate']

##### Quantitative metrics summary for QIN Whole prostate segment

In [85]:
print(f"nnunet task024 model DSC mean and std for whole prostate: {round(np.mean(nnnunet_task024_prostate_qin_df.dsc.values),2)}/{round(np.std(nnnunet_task024_prostate_qin_df.dsc.values),2)}")
print(f"nnunet task05 model DSC mean and std for whole prostate: {round(np.mean(nnnunet_task05_prostate_qin_df.dsc.values),2)}/{round(np.std(nnnunet_task05_prostate_qin_df.dsc.values),2)}")
print(f"nnunet monai prostate158 DSC model mean and std for whole prostate: {round(np.mean(monai_prostate158_prostate_qin_df.dsc.values),2)}/{round(np.std(monai_prostate158_prostate_qin_df.dsc.values),2)}")

nnunet task024 model DSC mean and std for whole prostate: 0.85/0.05
nnunet task05 model DSC mean and std for whole prostate: 0.67/0.16
nnunet monai prostate158 DSC model mean and std for whole prostate: 0.79/0.12


In [86]:
print(f"nnunet task024 model HSDFF95 mean and std for whole prostate: {round(np.mean(nnnunet_task024_prostate_qin_df.hsdff_95.values),2)}/{round(np.std(nnnunet_task024_prostate_qin_df.hsdff_95.values),2)}")
print(f"nnunet task05 model HSDFF95 mean and std for whole prostate: {round(np.mean(nnnunet_task05_prostate_qin_df.hsdff_95.values),2)}/{round(np.std(nnnunet_task05_prostate_qin_df.hsdff_95.values),2)}")
print(f"nnunet monai prostate158 DSC model mean and std for whole prostate: {round(np.mean(monai_prostate158_prostate_qin_df.hsdff_95.values),2)}/{round(np.std(monai_prostate158_prostate_qin_df.hsdff_95.values),2)}")

nnunet task024 model HSDFF95 mean and std for whole prostate: 2.72/2.02
nnunet task05 model HSDFF95 mean and std for whole prostate: 11.64/8.25
nnunet monai prostate158 DSC model mean and std for whole prostate: 5.61/7.34


In [87]:
nnnunet_task024_prostate_qin_df[['dsc', 'hsdff', 'hsdff_95', 'asd', 'ai_Value', 'expert_Value']].round(2).describe(include = 'all')

,dsc,hsdff,hsdff_95,asd,ai_Value,expert_Value
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,0.852667,10.483000,2.724333,2.038000,46.450000,46.449667
std,0.048490,4.300959,2.053337,0.921458,28.118539,28.112551
min,0.720000,6.070000,1.500000,0.960000,19.000000,19.040000
25%,0.822500,7.287500,1.660000,1.585000,25.025000,24.995000
50%,0.860000,9.595000,1.820000,1.785000,31.150000,31.150000
75%,0.890000,10.642500,3.037500,2.042500,62.200000,62.152500
max,0.920000,24.000000,9.760000,4.660000,115.900000,115.870000


In [88]:
nnnunet_task05_prostate_qin_df[['dsc', 'hsdff', 'hsdff_95', 'asd', 'ai_Value', 'expert_Value']].round(2).describe(include = 'all')

,dsc,hsdff,hsdff_95,asd,ai_Value,expert_Value
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,0.673000,30.489333,11.638667,5.681000,46.450000,46.449667
std,0.160262,14.721662,8.391738,3.664901,28.118539,28.112551
min,0.350000,6.310000,1.360000,1.090000,19.000000,19.040000
25%,0.547500,19.327500,3.250000,2.470000,25.025000,24.995000
50%,0.675000,29.395000,9.215000,5.245000,31.150000,31.150000
75%,0.815000,43.100000,20.360000,8.207500,62.200000,62.152500
max,0.900000,59.410000,26.230000,15.360000,115.900000,115.870000


In [89]:
monai_prostate158_prostate_qin_df[['dsc', 'hsdff', 'hsdff_95', 'asd', 'ai_Value', 'expert_Value']].round(2).describe(include = 'all')

,dsc,hsdff,hsdff_95,asd,ai_Value,expert_Value
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,0.792333,18.473000,5.612333,3.434000,46.450000,46.449667
std,0.125277,15.342885,7.465135,4.828262,28.118539,28.112551
min,0.230000,6.640000,1.570000,1.170000,19.000000,19.040000
25%,0.790000,9.022500,2.040000,1.700000,25.025000,24.995000
50%,0.830000,11.805000,3.365000,2.095000,31.150000,31.150000
75%,0.850000,20.277500,5.197500,2.560000,62.200000,62.152500
max,0.890000,72.650000,39.030000,26.770000,115.900000,115.870000


##### Paired tests on mean DSC, same collection == Wilcoxon signed rank test

Pair 1 : task024 dsc differences vs task05


In [93]:
diff = nnnunet_task024_prostate_qin_df.sort_values(by=['refSerieUID']).dsc.values - nnnunet_task05_prostate_qin_df.sort_values(by=['refSerieUID']).dsc.values
res = wilcoxon(diff)
res.statistic, res.pvalue

/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:4088: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


(0.0, 5.5887145476589365e-06)

In [94]:
res = wilcoxon(diff, alternative='greater')
res.statistic, res.pvalue

(378.0, 2.7943572738294682e-06)

Pair 2 : task024 dsc differences vs monai


In [97]:
diff = nnnunet_task024_prostate_qin_df.sort_values(by=['refSerieUID']).dsc.values - monai_prostate158_prostate_qin_df.sort_values(by=['refSerieUID']).dsc.values
res = wilcoxon(diff)
res.statistic, res.pvalue

/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:4088: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


(30.0, 4.876307514715551e-05)

In [98]:
res = wilcoxon(diff, alternative='greater')
res.statistic, res.pvalue

(405.0, 2.4381537573577754e-05)

Pair 3 : task05 dsc differences vs monai


In [99]:
diff = monai_prostate158_prostate_qin_df.sort_values(by=['refSerieUID']).dsc.values - nnnunet_task05_prostate_qin_df.sort_values(by=['refSerieUID']).dsc.values
res = wilcoxon(diff)
res.statistic, res.pvalue

/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:4088: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


(69.5, 0.001370044847525478)

In [100]:
res = wilcoxon(diff, alternative='less')
res.statistic, res.pvalue

(365.5, 0.9993149775762372)

#### Peripheral zone


##### Quantitative metrics summary for QIN Peripheral Zone segment

In [107]:
pz_qin_df = data_df[(data_df.collection_id == 'qin_prostate_repeatability') & (data_df.segment.str.contains('Peripheral'))]

In [110]:
pz_qin_df.algorithmNameCollection.unique()

array(['prostate158-qin_prostate_repeatability-prostate',
       'nnunet_task05-qin_prostate_repeatability-prostate'], dtype=object)

In [112]:
monai_prostate158_pz_qin_df = pz_qin_df[pz_qin_df.algorithmNameCollection == 'prostate158-qin_prostate_repeatability-prostate']
nnnunet_task05_pz_qin_df = pz_qin_df[pz_qin_df.algorithmNameCollection == 'nnunet_task05-qin_prostate_repeatability-prostate']

In [113]:
nnnunet_task05_pz_qin_df = nnnunet_task05_pz_qin_df.drop_duplicates(subset='refSerieUID', keep="last")
monai_prostate158_pz_qin_df = monai_prostate158_pz_qin_df.drop_duplicates(subset='refSerieUID', keep="last")

In [114]:
nnnunet_task05_pz_qin_df[['dsc', 'hsdff', 'hsdff_95', 'asd', 'ai_Value', 'expert_Value']].round(2).describe(include = 'all')

,dsc,hsdff,hsdff_95,asd,ai_Value,expert_Value
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,0.489000,18.677333,10.041333,5.182667,9.903333,9.896333
std,0.268166,13.402669,9.492406,5.766393,3.975310,3.982878
min,0.000000,6.000000,2.280000,0.830000,3.200000,3.200000
25%,0.225000,9.100000,3.027500,1.362500,6.700000,6.662500
50%,0.595000,11.315000,4.550000,1.845000,9.650000,9.655000
75%,0.725000,30.647500,17.895000,8.510000,13.000000,12.997500
max,0.800000,48.610000,31.330000,21.900000,18.600000,18.640000


In [115]:
monai_prostate158_pz_qin_df[['dsc', 'hsdff', 'hsdff_95', 'asd', 'ai_Value', 'expert_Value']].round(2).describe(include = 'all')

,dsc,hsdff,hsdff_95,asd,ai_Value,expert_Value
count,30.000000,30.000000,30.000000,30.00000,30.000000,30.000000
mean,0.632667,13.561667,5.083333,2.07600,9.903333,9.896333
std,0.159026,7.551380,4.430007,2.47489,3.975310,3.982878
min,0.160000,5.570000,1.850000,0.66000,3.200000,3.200000
25%,0.630000,8.962500,2.802500,1.04250,6.700000,6.662500
50%,0.680000,11.885000,3.445000,1.27000,9.650000,9.655000
75%,0.720000,14.985000,5.302500,1.91500,13.000000,12.997500
max,0.780000,38.090000,21.210000,11.31000,18.600000,18.640000


In [116]:
print(f"nnunet task05 model DSC mean and std for pz: {round(np.mean(nnnunet_task05_pz_qin_df.dsc.values),2)}/{round(np.std(nnnunet_task05_pz_qin_df.dsc.values),2)}")
print(f"nnunet monai prostate158 DSC model mean and std for pz: {round(np.mean(monai_prostate158_pz_qin_df.dsc.values),2)}/{round(np.std(monai_prostate158_pz_qin_df.dsc.values),2)}")

nnunet task05 model DSC mean and std for pz: 0.49/0.26
nnunet monai prostate158 DSC model mean and std for pz: 0.63/0.16


##### Paired tests on mean DSC, same collection == Wilcoxon signed rank test

In [117]:
diff = monai_prostate158_pz_qin_df.sort_values(by=['refSerieUID']).hsdff_95.values - nnnunet_task05_pz_qin_df.sort_values(by=['refSerieUID']).hsdff_95.values
res = wilcoxon(diff)
res.statistic, res.pvalue

(56.5, 0.00011058151721954346)

In [118]:
res = wilcoxon(diff, alternative='greater')
res.statistic, res.pvalue

(56.5, 0.9999504871666431)

### Prostate MRI US Biopsy


#### Whole prostate


In [119]:
prostate_mri_us_biopsy_df = data_df[(data_df.collection_id == 'prostate_mri_us_biopsy') & (data_df.segment == 'Prostate')]

In [120]:
prostate_mri_us_biopsy_df.algorithmNameCollection.unique()

array(['prostate158-prostate_mri_us_biopsy-prostate',
       'nnunet_task024-prostate_mri_us_biopsy-prostate'], dtype=object)

In [123]:
nnunet_task024_prostate_prostate_mri_us_biopsy_df = prostate_mri_us_biopsy_df[prostate_mri_us_biopsy_df.algorithmNameCollection \
    == 'nnunet_task024-prostate_mri_us_biopsy-prostate']
monai_prostate158_prostate_prostate_mri_us_biopsy_df = prostate_mri_us_biopsy_df[prostate_mri_us_biopsy_df.algorithmNameCollection \
    == 'prostate158-prostate_mri_us_biopsy-prostate']

##### Quantitative metrics summary for PROSTATE-MRI-US-Biopsy whole prostate segment

In [124]:
print(f"nnunet task024 model DSC mean and std for prostate: \
      {round(np.mean(nnunet_task024_prostate_prostate_mri_us_biopsy_df.dsc.values),2)}/{round(np.std(nnunet_task024_prostate_prostate_mri_us_biopsy_df.dsc.values),2)}")
print(f"nnunet monai prostate158 DSC model mean and std for prostate: \
      {round(np.mean(monai_prostate158_prostate_prostate_mri_us_biopsy_df.dsc.values),2)}/{round(np.std(monai_prostate158_prostate_prostate_mri_us_biopsy_df.dsc.values),2)}")

nnunet task024 model DSC mean and std for prostate:       0.86/0.09
nnunet monai prostate158 DSC model mean and std for prostate:       0.84/0.16


In [125]:
nnunet_task024_prostate_prostate_mri_us_biopsy_df[['dsc', 'hsdff', 'hsdff_95', 'asd', 'ai_Value', 'expert_Value']].round(2).describe(include = 'all')

,dsc,hsdff,hsdff_95,asd,ai_Value,expert_Value
count,1017.000000,1017.000000,1017.000000,1006.000000,1017.000000,1017.000000
mean,0.856893,16.565703,3.454444,6.564503,48.970403,48.969587
std,0.092948,23.927155,7.697978,6.581671,26.344915,26.345122
min,0.080000,3.600000,0.330000,1.130000,10.600000,10.560000
25%,0.840000,6.360000,1.080000,3.362500,31.700000,31.740000
50%,0.880000,8.090000,1.530000,4.990000,41.500000,41.490000
75%,0.910000,11.460000,2.300000,7.342500,59.800000,59.840000
max,0.950000,135.460000,58.030000,57.960000,222.400000,222.410000


In [126]:
monai_prostate158_prostate_prostate_mri_us_biopsy_df[['dsc', 'hsdff', 'hsdff_95', 'asd', 'ai_Value', 'expert_Value']].round(2).describe(include = 'all')

,dsc,hsdff,hsdff_95,asd,ai_Value,expert_Value
count,1017.000000,1017.000000,1017.000000,1006.000000,1017.000000,1017.000000
mean,0.840197,13.026912,4.598496,6.855070,48.970403,48.969587
std,0.157959,18.561501,10.963340,7.069255,26.344915,26.345122
min,0.000000,3.580000,0.330000,1.070000,10.600000,10.560000
25%,0.860000,6.180000,1.080000,3.270000,31.700000,31.740000
50%,0.890000,7.680000,1.500000,4.800000,41.500000,41.490000
75%,0.910000,9.680000,2.160000,7.295000,59.800000,59.840000
max,0.950000,132.990000,78.960000,71.780000,222.400000,222.410000


##### Paired tests on mean DSC, same collection == Wilcoxon signed rank test

In [127]:
diff = nnunet_task024_prostate_prostate_mri_us_biopsy_df.sort_values(by=['refSerieUID']).hsdff_95.values - \
    monai_prostate158_prostate_prostate_mri_us_biopsy_df.sort_values(by=['refSerieUID']).hsdff_95.values
res = wilcoxon(diff)
res.statistic, res.pvalue

(226905.0, 0.07506454031591767)

In [128]:
res = wilcoxon(diff, alternative='greater')
res.statistic, res.pvalue

(258700.0, 0.03753227015795883)